In [1]:
import numpy as np 
import pandas as pd 
import matplotlib as mpl
import matplotlib.pyplot as plt

import ray

from tools_ryu import mytool
from multi_run_v3.data_pp import DataPreprocess
from multi_run_v3.momentum import make_group_mask
from multi_run_v3.momentum_strategy import *
from multi_run_v2.initialize_v2 import make_market_index

2023-07-24 22:24:17,120	INFO worker.py:1625 -- Started a local Ray instance.


In [2]:
data = pd.read_pickle("Data/cmktcap_data_final.pickle")

days_lst = ["MON","TUE","WED","THU","FRI","SAT","SUN"]

In [3]:
data_pp = DataPreprocess()
data_pp.initialize(data=data)
data_pp.make_mask(mktcap_thresh=1_000_000,
                  vol_thresh=1_000_000,
                  ma=True)

In [4]:
mkt = make_market_index(data_pp.price_df,
                        data_pp.mktcap_df,
                        data_pp.vol_df,
                        1_000_000,
                        1_000_000,
                        True)

In [96]:
mkt_pf = (mkt + 1).cumprod()

weekly_rtn_dict = {}
for day in days_lst:
    weekly_rtn = mkt_pf.pct_change(7).resample(f'W-{day}').last()
    
    mean = weekly_rtn.iloc[1:].mean() 
    std = weekly_rtn.iloc[1:].std() 
    
    weekly_pf_rtn = (weekly_rtn.fillna(0) + 1).cumprod()
    peak = weekly_pf_rtn.cummax()
    weekly_mdd = ((weekly_pf_rtn - peak) / peak).min()
    
    weekly_rtn_dict[day] = {'mean':mean,
                            'std':std,
                            'MDD':weekly_mdd}

In [97]:
m = mkt.iloc[0:].mean() * 7
std = mkt.iloc[0:].std() * np.sqrt(7)

peak = mkt_pf.cummax()
daily_mdd = ((mkt_pf - peak) / peak).min()

In [100]:
df = pd.concat([pd.DataFrame(weekly_rtn_dict),pd.DataFrame([m, std, daily_mdd],
                  index=['mean', 'std', 'MDD'],
                 columns=['Daily']) ], axis=1)

df.loc["Sharpe",:] = df.iloc[0] / df.iloc[1]
df

,MON,TUE,WED,THU,FRI,SAT,SUN,Daily
mean,0.013090,0.012963,0.013097,0.012843,0.012648,0.013229,0.013083,0.012826
std,0.106533,0.105841,0.106766,0.104665,0.100750,0.106396,0.106974,0.103435
MDD,-0.868867,-0.869491,-0.869786,-0.878548,-0.886371,-0.891023,-0.891594,-0.892702
Sharpe,0.122870,0.122472,0.122669,0.122708,0.125539,0.124338,0.122304,0.123998
